# BT4Py HiTypeT5 Top-1

In [1]:
%load_ext autoreload
%autoreload 2

%pwd
%cd /home/benji/Documents/Uni/heidelberg/05/masterarbeit/impls/scripts/experiments

/home/benji/Documents/Uni/heidelberg/05/masterarbeit/impls/scripts/experiments


In [2]:
import polars as pl

pl.Config.set_fmt_str_lengths(300)
pl.Config.set_tbl_rows(n=50)

polars.config.Config

In [3]:
import pathlib

from scripts.common.schemas import TypeCollectionCategory
from scripts.infer.structure import DatasetFolderStructure

tool = "HiTypilusN1"
dataset = DatasetFolderStructure(pathlib.Path(
    "/home/benji/Documents/Uni/heidelberg/05/masterarbeit/datasets/better-types-4-py-dataset"
))


2023-07-27 15:35:57.084392: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-27 15:35:58.157504: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
import logging
from importlib import reload

logging.shutdown()
reload(logging)

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
for handler in logger.handlers:
    logger.removeHandler(handler)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

ch.setFormatter(logging.Formatter(f"[{tool} @ %(levelname)s]: %(message)s"))
logger.addHandler(ch)

logger.info("Hello World!")


[HiTypilusN1 @ INFO]: Hello World!


# Ensure Datapoints line up

# Join Datapoints based on file, category and qname_ssa

In [8]:
import polars as pl

queries_for_inner = []
queries_for_missing = []
for author_repo in tqdm.tqdm(ground_truths):
    ground_truth = pl.read_csv(source=ground_truths[author_repo], null_values=[""]).filter(
        pl.col(ExtendedInferredSchema.category).is_in(tasks_str)
    ).with_columns(pl.lit(str(author_repo)).alias("repository"))
    #print(ground_truth)

    inferred = pl.concat([
        pl.read_csv(source=inferreds[task][author_repo], null_values=[""])
        for task in tasks
    ]).with_columns(pl.lit(str(author_repo)).alias("repository"))
    #print(inferred)

    
    gt_vs_inf = ground_truth.join(
        inferred,
        on=[
            "repository",
            ExtendedInferredSchema.file,
            ExtendedInferredSchema.category,
            ExtendedInferredSchema.qname,
            ExtendedInferredSchema.qname_ssa,
        ],
        how="inner",
        # validate="1:1",
        suffix=("_predict"),
    )
    #print(gt_vs_inf)
    # print(gt_vs_inf)
    queries_for_inner.append(gt_vs_inf)


for author_repo, merged, _ in zip(
    ground_truths, 
    # pl.collect_all(queries_for_inner, streaming=True, common_subplan_elimination=False),
    queries_for_inner,
    range(3)
):
    for task in tasks_str:
        task_sample = (
            merged.select([
                pl.col("repository"),
                pl.col(ExtendedInferredSchema.file),
                pl.col(ExtendedInferredSchema.category),
                pl.col(ExtendedInferredSchema.qname_ssa),
                pl.col(ExtendedInferredSchema.anno),
                pl.col("anno_predict"),
                pl.col(ExtendedInferredSchema.parametric_anno),
                pl.col("parametric_anno_predict"),
            ]).filter(pl.col(ExtendedInferredSchema.category) == task).head(n=20))
        
        logger.info(f"{task}, {task_sample}")


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 396.16it/s]
[TypeT5TopN1 @ INFO]: CALLABLE_PARAMETER, shape: (20, 8)
┌──────────────────────────┬────────────────────┬────────────────────┬──────────────────────────────────────────────┬──────┬────────────────────────────────────────┬─────────────────┬────────────────────────────────────────┐
│ repository               ┆ file               ┆ category           ┆ qname_ssa                                    ┆ anno ┆ anno_predict                           ┆ parametric_anno ┆ parametric_anno_predict                │
│ ---                      ┆ ---                ┆ ---                ┆ ---                                          ┆ ---  ┆ ---                                    ┆ ---             ┆ ---                                    │
│ str                      ┆ 

[TypeT5TopN1 @ INFO]: VARIABLE, shape: (20, 8)
┌──────────────────────────┬────────────────────────────────────┬──────────┬───────────────────────────────────────────────────────┬──────┬──────────────┬─────────────────┬─────────────────────────┐
│ repository               ┆ file                               ┆ category ┆ qname_ssa                                             ┆ anno ┆ anno_predict ┆ parametric_anno ┆ parametric_anno_predict │
│ ---                      ┆ ---                                ┆ ---      ┆ ---                                                   ┆ ---  ┆ ---          ┆ ---             ┆ ---                     │
│ str                      ┆ str                                ┆ str      ┆ str                                                   ┆ str  ┆ str          ┆ str             ┆ str                     │
╞══════════════════════════╪════════════════════════════════════╪══════════╪═══════════════════════════════════════════════════════╪══════╪══════════════╪═══

[TypeT5TopN1 @ INFO]: CALLABLE_RETURN, shape: (20, 8)
┌────────────────────────┬──────────────────┬─────────────────┬─────────────────────────┬────────────────────────────────────────┬────────────────────────────────────────┬───────────────────┬─────────────────────────┐
│ repository             ┆ file             ┆ category        ┆ qname_ssa               ┆ anno                                   ┆ anno_predict                           ┆ parametric_anno   ┆ parametric_anno_predict │
│ ---                    ┆ ---              ┆ ---             ┆ ---                     ┆ ---                                    ┆ ---                                    ┆ ---               ┆ ---                     │
│ str                    ┆ str              ┆ str             ┆ str                     ┆ str                                    ┆ str                                    ┆ str               ┆ str                     │
╞════════════════════════╪══════════════════╪═════════════════╪═══════════

[TypeT5TopN1 @ INFO]: CALLABLE_PARAMETER, shape: (20, 8)
┌──────────────────┬─────────────────────────┬────────────────────┬────────────────────────────────────────────┬─────────────────────────────────────────┬─────────────────────────────────────────┬────────────────────────────┬────────────────────────────┐
│ repository       ┆ file                    ┆ category           ┆ qname_ssa                                  ┆ anno                                    ┆ anno_predict                            ┆ parametric_anno            ┆ parametric_anno_predict    │
│ ---              ┆ ---                     ┆ ---                ┆ ---                                        ┆ ---                                     ┆ ---                                     ┆ ---                        ┆ ---                        │
│ str              ┆ str                     ┆ str                ┆ str                                        ┆ str                                     ┆ str                    

[TypeT5TopN1 @ INFO]: VARIABLE, shape: (20, 8)
┌──────────────────┬──────────────────────────┬──────────┬────────────────────────────────────────────────────┬─────────────────────────────┬───────────────────────────────────┬─────────────────┬─────────────────────────┐
│ repository       ┆ file                     ┆ category ┆ qname_ssa                                          ┆ anno                        ┆ anno_predict                      ┆ parametric_anno ┆ parametric_anno_predict │
│ ---              ┆ ---                      ┆ ---      ┆ ---                                                ┆ ---                         ┆ ---                               ┆ ---             ┆ ---                     │
│ str              ┆ str                      ┆ str      ┆ str                                                ┆ str                         ┆ str                               ┆ str             ┆ str                     │
╞══════════════════╪══════════════════════════╪══════════╪═══════

# Check transformed dataset

In [68]:
logger.info(filtered_ground_truths_vs_predictions.select(
    pl.col("repository"),
    pl.col(ExtendedInferredSchema.file),
    pl.col(ExtendedInferredSchema.category),
    pl.col(ExtendedInferredSchema.qname_ssa),
    pl.col(ExtendedInferredSchema.anno),
    pl.col("rewritten_anno"),
).sample(n=20))

[TypeT5TopN1 @ INFO]: shape: (20, 6)
┌────────────────────────────────────────┬─────────────────────────────────────────────────────────────────┬────────────────────┬───────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────┬──────────────────────────────┐
│ repository                             ┆ file                                                            ┆ category           ┆ qname_ssa                                             ┆ anno                                                             ┆ rewritten_anno               │
│ ---                                    ┆ ---                                                             ┆ ---                ┆ ---                                                   ┆ ---                                                              ┆ ---                          │
│ str                                    ┆ str                                                             ┆ st

In [69]:
logger.info(filtered_ground_truths_vs_predictions.select(
    pl.col("repository"),
    pl.col(ExtendedInferredSchema.file),
    pl.col(ExtendedInferredSchema.category),
    pl.col(ExtendedInferredSchema.qname_ssa),
    pl.col(ExtendedInferredSchema.parametric_anno),
    pl.col("rewritten_parametric_anno"),
).sample(n=20))

[TypeT5TopN1 @ INFO]: shape: (20, 6)
┌────────────────────────────────────────┬────────────────────────────────────────────────────────────────┬────────────────────┬──────────────────────────────────────────────────────────────────────────┬──────────────────────────────────────────┬───────────────────────────┐
│ repository                             ┆ file                                                           ┆ category           ┆ qname_ssa                                                                ┆ parametric_anno                          ┆ rewritten_parametric_anno │
│ ---                                    ┆ ---                                                            ┆ ---                ┆ ---                                                                      ┆ ---                                      ┆ ---                       │
│ str                                    ┆ str                                                            ┆ str                ┆ str      

In [70]:
logger.info(filtered_ground_truths_vs_predictions.select(
    pl.col("repository"),
    pl.col(ExtendedInferredSchema.file),
    pl.col(ExtendedInferredSchema.category),
    pl.col(ExtendedInferredSchema.qname_ssa),
    pl.col("anno_predict"),
    pl.col("rewritten_anno_predict"),
).sample(n=20))

[TypeT5TopN1 @ INFO]: shape: (20, 6)
┌────────────────────────────────────────┬─────────────────────────────────────────────┬────────────────────┬─────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────┬────────────────────────────────┐
│ repository                             ┆ file                                        ┆ category           ┆ qname_ssa                               ┆ anno_predict                                                                            ┆ rewritten_anno_predict         │
│ ---                                    ┆ ---                                         ┆ ---                ┆ ---                                     ┆ ---                                                                                     ┆ ---                            │
│ str                                    ┆ str                                         ┆ str                ┆ str                         

In [71]:
logger.info(filtered_ground_truths_vs_predictions.select(
    pl.col("repository"),
    pl.col(ExtendedInferredSchema.file),
    pl.col(ExtendedInferredSchema.category),
    pl.col(ExtendedInferredSchema.qname_ssa),
    pl.col("parametric_anno_predict"),
    pl.col("rewritten_parametric_anno_predict"),
).sample(n=20))

[TypeT5TopN1 @ INFO]: shape: (20, 6)
┌──────────────────────────────────────┬─────────────────────────────────────────────────────┬────────────────────┬──────────────────────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────────────┬───────────────────────────────────┐
│ repository                           ┆ file                                                ┆ category           ┆ qname_ssa                                                                        ┆ parametric_anno_predict                                                  ┆ rewritten_parametric_anno_predict │
│ ---                                  ┆ ---                                                 ┆ ---                ┆ ---                                                                              ┆ ---                                                                      ┆ ---                               │
│ str                            

# Accept transformed dataset

In [72]:
filtered_ground_truths_vs_predictions = filtered_ground_truths_vs_predictions.with_columns([
    pl.col("rewritten_anno").alias(ExtendedInferredSchema.anno),
    pl.col("rewritten_parametric_anno").alias(ExtendedInferredSchema.parametric_anno),
    pl.col("rewritten_anno_predict").alias("anno_predict"),
    pl.col("rewritten_parametric_anno_predict").alias("parametric_anno_predict")
]).drop(["rewritten_anno", "rewritten_parametric_anno", "rewritten_anno_predict", "rewritten_parametric_anno_predict"])

In [73]:
logger.info(filtered_ground_truths_vs_predictions.select(
    pl.col("repository"),
    pl.col(ExtendedInferredSchema.file),
    pl.col(ExtendedInferredSchema.category),
    pl.col(ExtendedInferredSchema.qname_ssa),
    pl.col("anno"),
    pl.col("anno_predict"),
    pl.col("parametric_anno"),
    pl.col("parametric_anno_predict"),
).sample(n=20))

[TypeT5TopN1 @ INFO]: shape: (20, 8)
┌──────────────────────────────────────┬─────────────────────────────────────────────────────────────┬────────────────────┬─────────────────────────────────────────────────────────────────────────────────┬───────────────────────────────────┬───────────────────────────────────┬─────────────────────┬─────────────────────────┐
│ repository                           ┆ file                                                        ┆ category           ┆ qname_ssa                                                                       ┆ anno                              ┆ anno_predict                      ┆ parametric_anno     ┆ parametric_anno_predict │
│ ---                                  ┆ ---                                                         ┆ ---                ┆ ---                                                                             ┆ ---                               ┆ ---                               ┆ ---                 ┆ ---        

#  Prediction Metrics for Full Accuracy

In [14]:
import pathlib, shutil, tqdm

from typet5.static_analysis import PythonProject
from typet5.experiments.utils import remove_newer_syntax_for_repo
from typet5.experiments.typilus import TypilusSupportedSyntax

repos_dir = dataset.dataset_root
typilus_rewritten_repos = pathlib.Path() / "typilus-repos"
if not typilus_rewritten_repos.is_dir():
    logger.info("Creating rewritten repos")
    shutil.copytree(repos_dir / "repos" / "test", typilus_rewritten_repos)
    remove_newer_syntax_for_repo(typilus_rewritten_repos, TypilusSupportedSyntax)

logger.info(f"Rewritten repos are at {typilus_rewritten_repos.resolve()}")
name_to_location = {r.name: r for r in typilus_rewritten_repos.iterdir() if r.is_dir()}
print(len(name_to_location))

name_to_parsed = {
    name: PythonProject.parse_from_root(root, discard_bad_files=True) 
    for name, root in tqdm.tqdm(name_to_location.items(), "Parsing labels")
}
# print(name_to_parsed["ActivityWatch__aw-research"])

[HiTypilusN1 @ INFO]: Rewritten repos are at /home/benji/Documents/Uni/heidelberg/05/masterarbeit/impls/scripts/experiments/typilus-repos


50



Parsing labels: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [02:11<00:00,  2.63s/it]


In [21]:
# TypeT5 paper has Typilus evaluated ONLY on common types, as the freely trained model has no ability of predicting 
#
# The original Typilus model can predict from a set of common types as well as (nonparametric) userdefined types, 
# but their released model can only predict common types, so we only evaluate its performance on common types. - TypeT5 Paper
import libcst

from typet5.model import ModelWrapper
model = ModelWrapper.load_from_hub("MrVPlusOne/TypeT5-v7")
common_names = common_types = model.common_type_names

logger.debug(f"{len(common_types)} common types are known by TypeT5")
logger.debug(f"{common_types}")

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

[HiTypilusN1 @ DEBUG]: 100 common types are known by TypeT5
[HiTypilusN1 @ DEBUG]: {'IO', 'Collection', 'Qubit', 'bytes', 'Decimal', 'Table', 'DataFrame', 'Client', 'Set', 'ndarray', '_MakeClient', 'UserID', 'Item', 'T', 'object', '_T', 'PartyID', 'Parameter', 'Token', 'Scope', 'Flask', 'Container', 'MagicMock', 'Dict', 'Config', 'View', 'Module', 'None', 'HttpRequest', 'DataT', 'Variable', 'Text', 'Address', 'Source', 'BaseException', 'Path', 'Node', 'Type', 'User', 'Settings', 'type', 'ArgumentParser', 'Outcome', 'Tuple', 'Iterable', 'CWLObjectType', 'Tensor', 'Task', 'str', 'List', 'Name', 'URL', 'Logger', 'Result', 'Any', 'Model', 'Generator', 'int', 'Union', 'Mapping', 'GlobalState', 'Namespace', 'Configuration', 'Literal', '<List>', 'Vertex', 'Awaitable', 'Expr', 'Exception', 'Redis', 'Nvim', 'Article', 'Iterator', 'Field', 'BlockHeaderAPI', 'BytesIO', 'Application', 'UserContext', 'Session', 'State', 'Root', '...', 'Callable', 'float', 'Mock', 'Event', 'Issue', 'Candidates', 'Co

In [19]:
import pathlib, collections

# Because our analysis reviews more datapoints than these models actually regard, reuse TypeT5 metrics instead
def typet5_metrics_4_hitypilus(task: TypeCollectionCategory | str) -> None:
    def cleanup_predictions(predictions: dict) -> dict:
        # {'global@global': [{'category': 'local', 'name': '__all__', 'type': ['tuple[typing.Text]']}]}
        rewritten_predictions = collections.defaultdict(list)
        for scope, ps in predictions.items():
            for p in ps:
                rewritten_p = p.copy()
                rewritten_p["type"] = [ty.replace("typing.Text", "str") for ty in rewritten_p["type"]]
                
                # if rewritten_p["type"] or rewritten_p["category"] != "local":
                rewritten_predictions[scope].append(rewritten_p)
        
        return rewritten_predictions
    
    from scripts.common.output import InferenceArtifactIO

    test_set = dataset.test_set()

    proj2datasets = [
        (
            project,
            InferenceArtifactIO(
                artifact_root=pathlib.Path(),
                dataset=dataset,
                repository=project,
                tool_name=tool,
                task=task,
            ),
        )
        for project in test_set
    ]
    existing = dict(
        (project, artifact)
        for project, artifact in proj2datasets
        if artifact.full_location().exists()
    )
    
    from importlib import reload

    from libcst import helpers as h
    import tqdm


    from typet5.experiments import hityper, type4py
    from typet5.static_analysis import PythonProject, SignatureMap, AccuracyMetric, SignatureErrorAnalysis
    from typet5.experiments.typet5 import accs_as_table_row
    from typet5.visualization import pretty_print_dict
    import pprint
    reload(hityper)

    assignments = dict[str, SignatureMap]()
    projects = dict[str, PythonProject]()

    for (project, artifact) in tqdm.tqdm(existing.items(), desc=f"Loading predictions from {task}"):
        typilus_predictions, hitypilus_predictions, _ = artifact.read()
        
        # rewritten_project_location = name_to_location[project.name]
        rewritten_project_parsed = name_to_parsed[project.name]
        
        # pprint.pprint(hity4py_predictions)
        # print()
        
        sigmap = SignatureMap()
        for fname, mres in hitypilus_predictions.items():
            #print(f"{fname=}, {mres=}")
            
            mres_rewritten = cleanup_predictions(mres)
            #print(fname, mres_rewritten)

            # Removing first three path tokens from fname because they represent the temporary directory / tmp and tmpasdas
            root, tmp, tmpfolder, *project_parts = pathlib.Path(fname).parts       
            mname = PythonProject.rel_path_to_module_name(pathlib.Path(*project_parts))
            
            parser = hityper.HiTyperResponseParser(module=mname)
            sigmap.update(parser.parse(mres_rewritten))
        
            #print(f"Adding sigmap for {mname}")
        assignments[project.name] = sigmap
        
        #print(f"Registering labels for {rewritten_project_location.name}")
        projects[project.name] = rewritten_project_parsed
    
    label_signatures: dict[str, SignatureMap] = {
        project_name: {e.path: e.get_signature() for e in project.all_elems()}
        for project_name, project in projects.items()
    }
    
    pred_signatures = assignments
    
    print(f"{len(label_signatures)=}, {len(pred_signatures)=}")
    
    #for (pname, mname), o in zip(module_srcs, assignments):        
        # pred_signatures[pname].update(o)

    # print("LABEL SIGS:", "ActivityWatch__aw-research", label_signatures["ActivityWatch__aw-research"], "\n")
    # print("PRED SIGS:", "ActivityWatch__aw-research", pred_signatures["ActivityWatch__aw-research"], "\n")

    eval_result = type4py.Type4PyEvalResult(
        pred_maps=pred_signatures,
        label_maps=label_signatures,
    )
    
    metrics = AccuracyMetric.default_metrics(common_type_names=common_names)
    # acc_metric = AccuracyMetric(common_type_names=ubiq_names)

    n_annots = sum([e.get_signature().n_annots() for _, p in projects.items() for e in p.all_elems()])
    n_labels = sum([e.n_annotated() for lm in eval_result.label_maps.values() for e in lm.values()])
    
    logger.info(f"n_annots: {n_annots}, n_labels: {n_labels}")
    logger.info(f"Ratio: {n_labels / n_annots}")
    
    accs = {
        m.name: SignatureErrorAnalysis(
            eval_result.pred_maps,
            eval_result.label_maps,
            m,
            error_on_mismatched_signature=False,
        ).accuracies
        for m in metrics
    }
    accs_as_table_row(accs)
    pretty_print_dict(accs)

In [22]:
typet5_metrics_4_hitypilus(task="all")

Loading predictions from all: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:02<00:00, 19.15it/s]


len(label_signatures)=50, len(pred_signatures)=50


[HiTypilusN1 @ INFO]: n_annots: 30070, n_labels: 16520
[HiTypilusN1 @ INFO]: Ratio: 0.5493847688726305


[((ty'str', ty'CompileFn'), 307),
 ((ty'str', ty'int'), 109),
 ((ty'int', ty'str'), 81),
 ((ty'mypy.nodes.Context', ty'GeneratorContext'), 64),
 ((ty'str', ty'sym.Symbol'), 59),
 ((ty'mypy.nodes.Context', ty'AnalyzerContext'), 59),
 ((ty'Union[None, str]', ty'str'), 58),
 ((ty'rx.core.Observable', ty'int'), 50),
 ((ty'abilian.core.sqlalchemy.SQLAlchemy', ty'DatabaseSession'), 49),
 ((ty'str', ty'runtime.Namespace'), 46),
 ((ty'bool', ty'Union[None, bool]'), 46),
 ((ty'mock.MagicMock', ty'ptm.MockFixture'), 43),
 ((ty'None', ty'Union[None, str]'), 41),
 ((ty'None', ty'str'), 37),
 ((ty'str', ty'Union[None, str]'), 33),
 ((ty'str', ty'Path'), 33),
 ((ty'sslyze.server_connectivity_info.ServerConnectivityInfo',
   ty'ServerConnectivityInfo'),
  31),
 ((ty'argparse.ArgumentParser', ty'QueryParser'), 29),
 ((ty'List', ty'List[str]'), 27),
 ((ty'str', ty'uuid.UUID'), 24)]
[((ty'str', ty'int'), 109),
 ((ty'int', ty'str'), 81),
 ((ty'Union[None, str]', ty'str'), 58),
 ((ty'rx.core.Observable', 

In [9]:
typet5_metrics_4_hitt5(task=TypeCollectionCategory.VARIABLE)

Loading predictions from VARIABLE: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:02<00:00, 23.09it/s]


len(label_signatures)=50, len(pred_signatures)=50


[HiType4PyN1 @ INFO]: n_annots: 30070, n_labels: 16520
[HiType4PyN1 @ INFO]: Ratio: 0.5493847688726305


Accuracies on all types:
header:  ['full.all', 'calibrated.all', 'calibrated.simple', 'calibrated.complex', 'base.all']
45.09 & 48.55 & 51.66 & 21.78 & 54.87
Accuracies on common types:
header:  ['full.all', 'calibrated.all', 'calibrated.simple', 'calibrated.complex', 'base.all']
62.32 & 61.36 & 64.66 & 30.62 & 66.24
Accuracies on rare types:
header:  ['full.all', 'calibrated.all', 'calibrated.simple', 'calibrated.complex', 'base.all']
9.92 & 29.90 & 32.36 & 10.89 & 34.24


In [10]:
typet5_metrics_4_hitt5(task=TypeCollectionCategory.CALLABLE_RETURN)

Loading predictions from CALLABLE_RETURN: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:02<00:00, 23.66it/s]


len(label_signatures)=50, len(pred_signatures)=50


[HiType4PyN1 @ INFO]: n_annots: 30070, n_labels: 16520
[HiType4PyN1 @ INFO]: Ratio: 0.5493847688726305


Accuracies on all types:
header:  ['full.all', 'calibrated.all', 'calibrated.simple', 'calibrated.complex', 'base.all']
45.11 & 48.66 & 51.82 & 21.87 & 54.99
Accuracies on common types:
header:  ['full.all', 'calibrated.all', 'calibrated.simple', 'calibrated.complex', 'base.all']
62.36 & 61.54 & 64.84 & 31.05 & 66.41
Accuracies on rare types:
header:  ['full.all', 'calibrated.all', 'calibrated.simple', 'calibrated.complex', 'base.all']
9.91 & 29.92 & 32.47 & 10.68 & 34.28


In [11]:
typet5_metrics_4_hitt5(task=TypeCollectionCategory.CALLABLE_PARAMETER)

Loading predictions from CALLABLE_PARAMETER: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:02<00:00, 23.50it/s]


len(label_signatures)=50, len(pred_signatures)=50


[HiType4PyN1 @ INFO]: n_annots: 30070, n_labels: 16520
[HiType4PyN1 @ INFO]: Ratio: 0.5493847688726305


Accuracies on all types:
header:  ['full.all', 'calibrated.all', 'calibrated.simple', 'calibrated.complex', 'base.all']
45.11 & 48.66 & 51.82 & 21.87 & 54.99
Accuracies on common types:
header:  ['full.all', 'calibrated.all', 'calibrated.simple', 'calibrated.complex', 'base.all']
62.36 & 61.54 & 64.84 & 31.05 & 66.41
Accuracies on rare types:
header:  ['full.all', 'calibrated.all', 'calibrated.simple', 'calibrated.complex', 'base.all']
9.91 & 29.92 & 32.47 & 10.68 & 34.28


In [12]:
typet5_metrics_4_hity4py(task="all")

Loading predictions from all: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:04<00:00, 11.92it/s]


len(label_signatures)=50, len(pred_signatures)=50


[HiType4PyN1 @ INFO]: n_annots: 30070, n_labels: 16520
[HiType4PyN1 @ INFO]: Ratio: 0.5493847688726305


Accuracies on all types:
header:  ['full.all', 'calibrated.all', 'calibrated.simple', 'calibrated.complex', 'base.all']
45.09 & 48.55 & 51.66 & 21.78 & 54.87
Accuracies on common types:
header:  ['full.all', 'calibrated.all', 'calibrated.simple', 'calibrated.complex', 'base.all']
62.32 & 61.36 & 64.66 & 30.62 & 66.24
Accuracies on rare types:
header:  ['full.all', 'calibrated.all', 'calibrated.simple', 'calibrated.complex', 'base.all']
9.92 & 29.90 & 32.36 & 10.89 & 34.24


In [ ]:
from typet5.static_analysis import SignatureErrorAnalysis, AccuracyMetric
from typet5.experiments.typet5 import accs_as_table_row
# from typet5.visualization import pretty_print_dict





accs_as_table_row(accs)
# pretty_print_dict(accs)


In [ ]:
from sklearn import metrics, preprocessing
import numpy as np

logger.info(f"Computing metrics on {len(filtered_ground_truths_vs_predictions)} samples")


logger.info("=== Adjusted Accuracy on All Types ===")
typet5_metrics(filtered_ground_truths_vs_predictions, column=ExtendedInferredSchema.anno, simple_complex=None)

logger.info("=== Adjusted Accuracy on Simple Types ===")
typet5_metrics(filtered_ground_truths_vs_predictions, column=ExtendedInferredSchema.anno, simple_complex="simple")

logger.info("=== Adjusted Accuracy on Complex Types ===")
typet5_metrics(filtered_ground_truths_vs_predictions, column=ExtendedInferredSchema.anno, simple_complex="complex")


In [76]:
from sklearn import metrics, preprocessing
import numpy as np

logger.info(f"Computing metrics on {len(filtered_ground_truths_vs_predictions)} samples")


logger.info("=== Base Accuracy on All Types ===")
typet5_metrics(filtered_ground_truths_vs_predictions, column=ExtendedInferredSchema.parametric_anno, simple_complex=None)


[TypeT5TopN1 @ INFO]: Computing metrics on 9629 samples
[TypeT5TopN1 @ INFO]: === Base Accuracy on All Types ===
[TypeT5TopN1 @ INFO]: accuracy=0.7751583757399523
[TypeT5TopN1 @ INFO]: recall=0.5992065365372685
[TypeT5TopN1 @ INFO]: precision=0.6050543966959154
[TypeT5TopN1 @ INFO]: f1score=0.5794400364724952


In [14]:
from sklearn import metrics, preprocessing

builtin_types = np.unique(filtered_ground_truths_vs_predictions.filter(
    pl.col(ExtendedInferredSchema.parametric_anno).str.starts_with("builtins.")
).get_column(ExtendedInferredSchema.parametric_anno).to_numpy())
print(builtin_types)
display = metrics.ConfusionMatrixDisplay.from_predictions(y_true=groundtruth_labels, y_pred=model_predictions, labels=builtin_types, normalize="true")

['builtins.Exception' 'builtins.UnicodeError' 'builtins.bool'
 'builtins.bytearray' 'builtins.bytes' 'builtins.complex' 'builtins.dict'
 'builtins.float' 'builtins.int' 'builtins.list' 'builtins.object'
 'builtins.set' 'builtins.str' 'builtins.tuple' 'builtins.type']


NameError: name 'groundtruth_labels' is not defined

In [ ]:
mock_mistakes = filtered_ground_truths_vs_predictions.select(pl.all()).filter(pl.col(ExtendedInferredSchema.parametric_anno).str.ends_with("Mock"))
#print(mock_mistakes.select(
#    pl.col("repository"),
#    pl.col(ExtendedInferredSchema.file),
#    pl.col(ExtendedInferredSchema.qname_ssa),
#    pl.col(ExtendedInferredSchema.parametric_anno),
#    pl.col("parametric_anno_predict"),
#))

groundtruth_labels = mock_mistakes.get_column(ExtendedInferredSchema.parametric_anno).to_numpy()
model_predictions = mock_mistakes.get_column("parametric_anno_predict").to_numpy()


display = metrics.ConfusionMatrixDisplay.from_predictions(
    y_true=groundtruth_labels, 
    y_pred=model_predictions, 
    labels=np.unique(np.concatenate((groundtruth_labels, model_predictions))), 
    normalize="true"
)

In [ ]:
#  Prediction Metrics for Full Accuracy